# 2021年7日打卡营大作业

大家好，这里是2021年7日打卡营大作业，本次作业内容为**实现文本相似度任务**，通过课上所学知识，实现文本相似度任务的代码。**目前已经给出了基于SimilarityNet的相似度任务的实现代码，同学们可以基于本代码进行修改或者重写，实现更高的准确率**，其中数据集已经上传，如果有误操，可以联系qq群助教，获得数据集。

## 【作业内容】

按照本项目给出的模块完成代码并跑通。

## 【作业提交】
我们会给出一个数据集压缩包，包含`baidu_train.tsv`,`baidu_dev.tsv`,`vocab.txt`和`test_forstu.tsv`。其中`baidu_train.tsv`,`baidu_dev.tsv`,`vocab.txt`用来训练模型。最后大家根据`test_forstu.tsv`测试数据输出预测结果。命名为`result.tsv`的文件并提交。
>note:遇到问题请及时向qq群的助教反馈。

## 【评分标准】

**优秀学员：**

条件：
1. 完成每天的课后作业；
2. 大作业代码运行成功且有结果；
3. 准确率排名前26名。

奖品设置：
|等级|奖品|名额|
|---|---|---|
|一等奖|小度在家|1|
|二等奖|耳机 |5|
|三等奖|书 | 10|
|四等奖|健身包|10|
|鼓励奖|招财熊|170|

# 1. 任务介绍
## 1.1 任务内容

文本语义匹配是自然语言处理中一个重要的基础问题，NLP领域的很多任务都可以抽象为文本匹配任务。例如，信息检索可以归结为查询项和文档的匹配，问答系统可以归结为问题和候选答案的匹配，对话系统可以归结为对话和回复的匹配。语义匹配在搜索优化、推荐系统、快速检索排序、智能客服上都有广泛的应用。如何提升文本匹配的准确度，是自然语言处理领域的一个重要挑战。

+ 信息检索：在信息检索领域的很多应用中，都需要根据原文本来检索与其相似的其他文本，使用场景非常普遍。
+ 新闻推荐：通过用户刚刚浏览过的新闻标题，自动检索出其他的相似新闻，个性化地为用户做推荐，从而增强用户粘性，提升产品体验。
+ 智能客服：用户输入一个问题后，自动为用户检索出相似的问题和答案，节约人工客服的成本，提高效率。

## 1.2 什么是文本匹配？

让我们来看一个简单的例子，比较各候选句子哪句和原句语义更相近

原句：“车头如何放置车牌”

+ 比较句1：“前牌照怎么装”
+ 比较句2：“如何办理北京车牌”
+ 比较句3：“后牌照怎么装”

（1）比较句1与原句，虽然句式和语序等存在较大差异，但是所表述的含义几乎相同

（2）比较句2与原句，虽然存在“如何” 、“车牌”等共现词，但是所表述的含义完全不同

（3）比较句3与原句，二者讨论的都是如何放置车牌的问题，只不过一个是前牌照，另一个是后牌照。二者间存在一定的语义相关性。

所以语义相关性，句1大于句3，句3大于句2.这就是语义匹配。


## 1.3 短文本语义匹配网络

短文本语义匹配(SimilarityNet, SimNet)是一个计算短文本相似度的框架，可以根据用户输入的两个文本，计算出相似度得分。主要包括BOW、CNN、RNN、MMDNN等核心网络结构形式，提供语义相似度计算训练和预测框架，适用于信息检索、新闻推荐、智能客服等多个应用场景，帮助企业解决语义匹配问题。

SimNet模型结构如图所示，包括输入层、表示层以及匹配层。

<br></br>
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/953b573d100b46fe84f026c99a3a2664fbecbe680fa748eda6337548a2f4b4af" width=50%></center>
<center><br>图 SimilarityNet 框架 </br></center>
<br></br>


## 1.4 实验设计

本任务给出一个文本相似度任务实现方法，模型框架结构图如下图所示，其中query和title是数据集经过处理后的待匹配的文本，然后经过分词处理，编码成id，经过SimilarityNet处理，得到输出，训练的损失函数使用的是交叉熵损失。

<br></br>
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/df8c743f9b3742e4a99ae6b066c133c9b119f7499e3b4addb5ce755ec3faae7e" width='600px'></center>
<center><br>图 SimilarityNet实验流程 </br></center>
<br></br>


# 2. 实验详细实现
文本相似度实验流程包含如下6个步骤：
1. **数据读取**：根据网络接收的数据格式，完成相应的预处理操作，保证模型正常读取；
2. **模型构建**：设计基于GRU的文本相似度模型，判断两句话是否是相似；
3. **训练配置**：实例化模型，选择模型计算资源（CPU或者GPU），指定模型迭代的优化算法；
4. **模型训练与保存**：执行多轮训练不断调整参数，以达到较好的效果，保存模型；
5. **模型评估**：训练好的模型在测试集合上测试；
6. **模型预测**：加载训练好的模型，并进行预测；

## 2.1数据读取

本实验共计需要读取四份数据: 训练集 **train.tsv**、验证集 **dev.tsv**、测试集 **test.tsv** 和 词汇表 **vocab.txt**。加载数据的代码如下：

+ 加载第三方库，paddle和paddlenlp相关的库

In [1]:
import math
import numpy as np
import os
import collections
from functools import partial
import random
import time
import inspect
import importlib
from tqdm import tqdm

import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import IterableDataset
from paddle.utils.download import get_path_from_url

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


+ 本实验需要依赖与paddlenlp，aistudio上的paddlenlp版本过低，所以需要首先升级paddlenlp

In [2]:
!pip install paddlenlp --upgrade

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 430kB 12.6MB/s eta 0:00:01
     |████████████████████████████████| 112kB 52.1MB/s eta 0:00:01
  Found existing installation: paddlenlp 2.0.0rc7
    Uninstalling paddlenlp-2.0.0rc7:
      Successfully uninstalled paddlenlp-2.0.0rc7


+ 导入paddlenlp相关的包

In [3]:
import paddlenlp as ppnlp
from paddlenlp.data import JiebaTokenizer, Pad, Stack, Tuple, Vocab
# from utils import convert_example
from paddlenlp.datasets import MapDataset
from paddle.dataset.common import md5file
from paddlenlp.datasets import DatasetBuilder

In [4]:
def convert_example(example, tokenizer, is_test=False):
    """
    为序列分类任务从序列生成模型输入
    """
    
    query, title = example["query"], example["title"]
    # query id生成
    query_ids = np.array(tokenizer.encode(query), dtype="int64")
    query_seq_len = np.array(len(query_ids), dtype="int64")

    # title id 生成
    title_ids = np.array(tokenizer.encode(title), dtype="int64")
    title_seq_len = np.array(len(title_ids), dtype="int64")
    
    # 训练模式
    if not is_test:
        label = np.array(example["label"], dtype="int64")
        return query_ids, title_ids, query_seq_len, title_seq_len, label
    else:
        # 测试模式
        return query_ids, title_ids, query_seq_len, title_seq_len

In [5]:
class BAIDUData(DatasetBuilder):

    SPLITS = {
        'train':os.path.join( 'baidu_train.tsv'),
        'dev': os.path.join('baidu_dev.tsv'),
    }

    def _get_data(self, mode, **kwargs):
        filename = self.SPLITS[mode]
        return filename

    def _read(self, filename):
        """读取数据"""
        with open(filename, 'r', encoding='utf-8') as f:
            head = None
            for line in f:
                data = line.strip().split("\t")
                if not head:
                    head = data
                else:
                    query, title, label = data
                    yield {"query": query, "title": title, "label": label}

    def get_labels(self):
        return ["0", "1"]

In [6]:
def load_dataset(name=None,
                 data_files=None,
                 splits=None,
                 lazy=None,
                 **kwargs):
   
    reader_cls = BAIDUData
    print(reader_cls)
    if not name:
        reader_instance = reader_cls(lazy=lazy, **kwargs)
    else:
        reader_instance = reader_cls(lazy=lazy, name=name, **kwargs)

    datasets = reader_instance.read_datasets(data_files=data_files, splits=splits)
    return datasets

+ 加载词汇表

In [7]:

vocab_path="vocab.txt"

vocab = Vocab.load_vocabulary(vocab_path, unk_token='[UNK]', pad_token='[PAD]')

+ 加载训练集，验证集，测试集

In [8]:
# Loads dataset.
train_ds, dev_ds = load_dataset(splits=["train", "dev"])

<class '__main__.BAIDUData'>


2021-06-08 19:08:47,165 - INFO - unique_endpoints {''}


+ 创建dataloader

In [13]:
def create_dataloader(dataset,
                      trans_fn=None,
                      mode='train',
                      batch_size=1, #1
                      batchify_fn=None):
    if trans_fn:
        dataset = dataset.map(trans_fn)

    shuffle = True if mode == 'train' else False
    if mode == "train":
        sampler = paddle.io.DistributedBatchSampler(
            dataset=dataset, batch_size=batch_size, shuffle=True)
    else:
        sampler = paddle.io.BatchSampler(
            dataset=dataset, batch_size=batch_size, shuffle=shuffle)
    dataloader = paddle.io.DataLoader(
        dataset,
        batch_sampler=sampler,
        return_list=True,
        collate_fn=batchify_fn)
    return dataloader

In [14]:
# Reads data and generates mini-batches.
batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=vocab.token_to_idx.get('[PAD]', 0)),  # query_ids
        Pad(axis=0, pad_val=vocab.token_to_idx.get('[PAD]', 0)),  # title_ids
        Stack(dtype="int64"),  # query_seq_lens
        Stack(dtype="int64"),  # title_seq_lens
        Stack(dtype="int64")  # label
    ): [data for data in fn(samples)]
tokenizer = ppnlp.data.JiebaTokenizer(vocab)

trans_fn = partial(convert_example, tokenizer=tokenizer, is_test=False)

## 2.2 模型构建

本节实验构建了基于GRU的SimilarityNet网络结构，如图所示。模型得输入是query和title，然后分别经过embedding层，GRU层，把GRU输出得向量拼接（concat操作），最后经过FC(全连接)层。
<br></br>
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/645678222ebb4c87bcf8b53e4e4ecbdc64340506839e498c8a3ed12fe57c0686" width='500px'></center>
<center><br>图 基于GRU的SimNet网络结构 </br></center>
<br></br>

In [15]:
class GRUEncoder(nn.Layer):

    def __init__(self,
                 input_size,
                 hidden_size,
                 num_layers=1,
                 direction="forward",
                 dropout=0.0,
                 pooling_type=None,
                 **kwargs):
        super().__init__()
        # 输入的大小
        self._input_size = input_size
        # 隐藏层的大小
        self._hidden_size = hidden_size
        self._direction = direction
        self._pooling_type = pooling_type
        # 实例化GRU层
        self.gru_layer = nn.GRU(input_size=input_size,
                                hidden_size=hidden_size,
                                num_layers=num_layers,
                                direction=direction,
                                dropout=dropout,
                                **kwargs)

    def get_input_dim(self):
        """
        Returns the dimension of the vector input for each element in the sequence input
        to a `GRUEncoder`. This is not the shape of the input tensor, but the
        last element of that shape.
        """
        return self._input_size

    def get_output_dim(self):
        """
        Returns the dimension of the final vector output by this `GRUEncoder`.  This is not
        the shape of the returned tensor, but the last element of that shape.
        """
        if self._direction == "bidirect":
            return self._hidden_size * 2
        else:
            return self._hidden_size

    def forward(self, inputs, sequence_length):
        
        encoded_text, last_hidden = self.gru_layer(
            inputs, sequence_length=sequence_length)
        if not self._pooling_type:
            # We exploit the `last_hidden` (the hidden state at the last time step for every layer)
            # to create a single vector.
            # If gru is not bidirection, then output is the hidden state of the last time step 
            # at last layer. Output is shape of `(batch_size, hidden_size)`.
            # If gru is bidirection, then output is concatenation of the forward and backward hidden state 
            # of the last time step at last layer. Output is shape of `(batch_size, hidden_size*2)`.
            if self._direction != 'bidirect':
                output = last_hidden[-1, :, :]
            else:
                output = paddle.concat(
                    (last_hidden[-2, :, :], last_hidden[-1, :, :]), axis=1)
        else:
            # We exploit the `encoded_text` (the hidden state at the every time step for last layer)
            # to create a single vector. We perform pooling on the encoded text.
            # The output shape is `(batch_size, hidden_size*2)` if use bidirectional GRU, 
            # otherwise the output shape is `(batch_size, hidden_size*2)`.
            if self._pooling_type == 'sum':
                output = paddle.sum(encoded_text, axis=1)
            elif self._pooling_type == 'max':
                output = paddle.max(encoded_text, axis=1)
            elif self._pooling_type == 'mean':
                output = paddle.mean(encoded_text, axis=1)
            else:
                raise RuntimeError(
                    "Unexpected pooling type %s ."
                    "Pooling type must be one of sum, max and mean." %
                    self._pooling_type)
        return output

In [16]:
class GRUModel(nn.Layer):
    def __init__(self,
                 vocab_size,
                 num_classes,
                 emb_dim=128,
                 padding_idx=0,
                 gru_hidden_size=128,
                 direction='forward',
                 gru_layers=1,
                 dropout_rate=0.0,
                 pooling_type=None,
                 fc_hidden_size=96):
        super().__init__()
        # 词嵌入层
        self.embedder = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
            padding_idx=padding_idx)
        # GRUb编码层
        self.gru_encoder = GRUEncoder(
            emb_dim,
            gru_hidden_size,
            num_layers=gru_layers,
            direction=direction,
            dropout=dropout_rate)
        # 线性层
        self.fc = nn.Linear(self.gru_encoder.get_output_dim() * 2,fc_hidden_size)
        # 输出层
        self.output_layer = nn.Linear(fc_hidden_size, num_classes)

    def forward(self, query, title, query_seq_len, title_seq_len):
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_query = self.embedder(query)
        embedded_title = self.embedder(title)
        # Shape: (batch_size, gru_hidden_size)
        query_repr = self.gru_encoder(
            embedded_query, sequence_length=query_seq_len)
        title_repr = self.gru_encoder(
            embedded_title, sequence_length=title_seq_len)
        # Shape: (batch_size, 2*gru_hidden_size)
        contacted = paddle.concat([query_repr, title_repr], axis=-1)
        # Shape: (batch_size, fc_hidden_size)
        fc_out = paddle.tanh(self.fc(contacted))
        # Shape: (batch_size, num_classes)
        logits = self.output_layer(fc_out)
        # probs = F.softmax(logits, axis=-1)

        return logits

In [17]:
class SimNet(nn.Layer):
    def __init__(self,
                 network,
                 vocab_size,
                 num_classes,
                 emb_dim=128,
                 pad_token_id=0):
        super().__init__()
        # 转换成小写
        network = network.lower()
        # gru网络
        if network == 'gru':
            self.model = GRUModel(
                vocab_size,
                num_classes,
                emb_dim,
                direction='forward',
                padding_idx=pad_token_id)

    def forward(self, query, title, query_seq_len=None, title_seq_len=None):
        logits = self.model(query, title, query_seq_len, title_seq_len)
        return logits

## 2.3 训练配置

In [18]:
# 检测是否可以使用GPU，如果可以优先使用GPU
use_gpu = True if paddle.get_device().startswith("gpu") else False
if use_gpu:
    paddle.set_device('gpu')
    #print('gpu')
batch_size=200   # batch size 的大小

+ 实例化SimilarityNet网络

In [19]:
# 构建SimNet网络，网络的主体采用GRUb网络
network='gru'
model = SimNet(
        network=network,
        vocab_size=len(vocab),
        num_classes=len(train_ds.label_list))
model = paddle.Model(model)

+ 构建训练集，验证集和测试集的dataloader

In [20]:
# 构建训练集的dataloader
train_loader = create_dataloader(
        train_ds,
        trans_fn=trans_fn,
        batch_size=batch_size,
        mode='train',
        batchify_fn=batchify_fn)
# 构建验证集的dataloader
dev_loader = create_dataloader(
        dev_ds,
        trans_fn=trans_fn,
        batch_size=batch_size,
        mode='validation',
        batchify_fn=batchify_fn)
# # 构建测试集的dataloader
# test_loader = create_dataloader(
#         test_ds,
#         trans_fn=trans_fn,
#         batch_size=batch_size,
#         mode='test',
#         batchify_fn=batchify_fn)

+ 定义优化器

In [21]:
lr=5e-4
optimizer = paddle.optimizer.Adam(
        parameters=model.parameters(), learning_rate=lr)

+ 实例化损失函数

In [22]:
# Defines loss and metric.
criterion = paddle.nn.CrossEntropyLoss()  # 交叉熵损失函数

+ 实例化评估方式

In [23]:
metric = paddle.metric.Accuracy()  # accuracy 评估方式

+ 编译模型

In [24]:
model.prepare(optimizer, criterion, metric)

## 2.4 模型训练

+ 模型训练使用paddle的高级API，使用fit函数就可以实现训练和模型保存，模型保存的位置为save_dir

In [25]:
epochs=10
save_dir='./checkpoints'
# Starts training and evaluating.
model.fit(
        train_loader,
        dev_loader,
        epochs=epochs,
        save_dir=save_dir
        )

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/10


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step  10/313 - loss: 0.6779 - acc: 0.5560 - 71ms/step
step  20/313 - loss: 0.6775 - acc: 0.5620 - 54ms/step
step  30/313 - loss: 0.6709 - acc: 0.5600 - 49ms/step
step  40/313 - loss: 0.6698 - acc: 0.5779 - 46ms/step
step  50/313 - loss: 0.6427 - acc: 0.5852 - 44ms/step
step  60/313 - loss: 0.6441 - acc: 0.5972 - 43ms/step
step  70/313 - loss: 0.6351 - acc: 0.6081 - 42ms/step
step  80/313 - loss: 0.5961 - acc: 0.6187 - 42ms/step
step  90/313 - loss: 0.5682 - acc: 0.6259 - 42ms/step
step 100/313 - loss: 0.6386 - acc: 0.6305 - 41ms/step
step 110/313 - loss: 0.5877 - acc: 0.6357 - 41ms/step
step 120/313 - loss: 0.5476 - acc: 0.6404 - 41ms/step
step 130/313 - loss: 0.5689 - acc: 0.6439 - 40ms/step
step 140/313 - loss: 0.5621 - acc: 0.6487 - 40ms/step
step 150/313 - loss: 0.5484 - acc: 0.6526 - 40ms/step
step 160/313 - loss: 0.5292 - acc: 0.6560 - 40ms/step
step 170/313 - loss: 0.5861 - acc: 0.6589 - 39ms/step
step 180/313 - loss: 0.5605 - acc: 0.6619 - 39ms/step
step 190/313 - loss: 0.5725 

## 2.5 模型评估

+ 模型评估实用paddle高级API自带的evaluate函数就可以实现评估，评估的方式是accuracy

In [24]:
# Finally tests model.
results = model.evaluate(dev_loader)
print("Finally test acc: %.5f" % results['acc'])

Eval begin...
step  10/122 - loss: 0.4908 - acc: 0.7703 - 24ms/step
step  20/122 - loss: 0.4314 - acc: 0.7695 - 18ms/step
step  30/122 - loss: 0.4691 - acc: 0.7760 - 16ms/step
step  40/122 - loss: 0.5401 - acc: 0.7867 - 15ms/step
step  50/122 - loss: 0.5670 - acc: 0.7866 - 15ms/step
step  60/122 - loss: 0.4395 - acc: 0.7878 - 14ms/step
step  70/122 - loss: 0.5325 - acc: 0.7895 - 14ms/step
step  80/122 - loss: 0.5728 - acc: 0.7879 - 14ms/step
step  90/122 - loss: 0.4095 - acc: 0.7905 - 13ms/step
step 100/122 - loss: 0.4865 - acc: 0.7930 - 13ms/step
step 110/122 - loss: 0.4447 - acc: 0.7923 - 13ms/step
step 120/122 - loss: 0.5081 - acc: 0.7926 - 13ms/step
step 122/122 - loss: 0.5163 - acc: 0.7918 - 13ms/step
Eval samples: 7802
Finally test acc: 0.79185


## 2.6 模型预测

In [26]:
def predict(model, data, label_map, batch_size=1, pad_token_id=0):
    """
    预测数据的标签
    """
    # 把数据分成若干个batch
    batches = [
        data[idx:idx + batch_size] for idx in range(0, len(data), batch_size)
    ]

    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=pad_token_id),  # query_ids
        Pad(axis=0, pad_val=pad_token_id),  # title_ids
        Stack(dtype="int64"),  # query_seq_lens
        Stack(dtype="int64"),  # title_seq_lens
    ): [data for data in fn(samples)]

    results = []
    model.eval()
    for batch in batches:
        query_ids, title_ids, query_seq_lens, title_seq_lens = batchify_fn(
            batch)
        query_ids = paddle.to_tensor(query_ids)
        title_ids = paddle.to_tensor(title_ids)
        query_seq_lens = paddle.to_tensor(query_seq_lens)
        title_seq_lens = paddle.to_tensor(title_seq_lens)
        logits = model(query_ids, title_ids, query_seq_lens, title_seq_lens)
        probs = F.softmax(logits, axis=1)
        idx = paddle.argmax(probs, axis=1).numpy()
        idx = idx.tolist()
        labels = [label_map[i] for i in idx]
        results.extend(labels)
    return results

+ 建立id到类别的映射，本实验有两个类别，编码为0，1。0类为dissimilar，1类为similar

In [27]:
label_map = {0: '0', 1: '1'}

+ 加载训练的模型的参数

In [28]:
# 实例化SimNet模型
model = SimNet(network=network, vocab_size=len(vocab), num_classes=len(label_map))
params_path='./checkpoints/final.pdparams'
#  加载模型参数
state_dict = paddle.load(params_path)
model.set_dict(state_dict)
print("Loaded parameters from %s" % params_path)

Loaded parameters from ./checkpoints/final.pdparams


In [29]:
def preprocess_prediction_data(data, tokenizer):
    examples = []
    for query, title in data:
        query_ids = tokenizer.encode(query)
        title_ids = tokenizer.encode(title)
        examples.append([query_ids, title_ids, len(query_ids), len(title_ids)])
    return examples

In [30]:
# Firstly pre-processing prediction data  and then do predict.
# 将data替换为我们给出的测试集test_forstu.tsv，输出预测结果result.tsv并提交。
import pandas as pd
data = [
        ['世界上什么东西最小', '世界上什么东西最小？'],
        ['光眼睛大就好看吗', '眼睛好看吗？'],
        ['小蝌蚪找妈妈怎么样', '小蝌蚪找妈妈是谁画的'],
    ]
data = pd.read_csv('test_forstu.tsv', sep='\t')
data = data.values.tolist()
examples = preprocess_prediction_data(data, tokenizer)

results = predict(
        model,
        examples,
        label_map=label_map,
        batch_size=batch_size,
        pad_token_id=vocab.token_to_idx.get('[PAD]', 0))

list1 = []
list2 = []
list3 = []

for idx, text in enumerate(data):
    list1.append(text[0])
    list2.append(text[1])
    list3.append(results[idx])
dict= {'text_a': list1, 'text_b': list2, 'label': list3}
datas = pd.DataFrame(dict)
datas.to_csv('result.csv',encoding='UTF-8',sep="\t")
    #print('Data: {} \t{}\t Label: {}'.format(text[0],text[1], results[idx]))